In [1]:
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime

from typing import Callable, List

from psycopg import sql

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch
from dmp.postgres_interface.element.column import Column
from dmp.postgres_interface.element.table import Table
from dmp.postgres_interface.element.column_group import ColumnGroup

from dmp.util.butter_e_export import *

pd.options.display.max_seq_items = None

%load_ext autoreload
%autoreload 2
credentials = jobqueue.load_credentials("dmp")


2023-11-17 07:52:48.842089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TODO: convert this into two steps:
    1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

          SELECT
                  {run}.{run_id}
          FROM
                  {run},
                  {job_status},
                  {job_data}
          WHERE TRUE
                  AND {run}.batch like {pattern}
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
              AND {job_status}.status = 2
          ORDER BY experiment_id, run_id;

    2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
      -> partition by the attributes we care about querying by (dataset, size, shape, depth)

              SELECT
                  {columns}
              FROM
                  {run},
                  {job_status},
                  {job_data}
              WHERE TRUE
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
                  AND {job_status}.status = 2
                  AND {run}.{run_id} IN ({ids})

        pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
        results = pool.uimap(download_chunk, chunks)
        for num_rows, chunk in results:
            num_stored += 1
            print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

    extra credit) extract butter data matching this as well into a new dataset
    extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
    

In [2]:
from psycopg import ClientCursor


print(f"run vars {vars(run)}")

columns = (
    run
    + ColumnGroup(*[c for c in job_status.columns if c.name != "id"])
    + job_data.command
)
print(columns.names)


def passthrough(row, index, value, column, data):
    data[column.name] = value


column_converters: List[Callable] = [passthrough for _ in columns]


def flatten_json(json_obj, destination=None, parent_key="", separator="_"):
    if isinstance(destination, dict):
        flattened = destination
    else:
        flattened = {}

    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


column_converters[
    columns.get_index_of(job_data.command)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)
column_converters[
    columns.get_index_of(run.run_data)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)


def parquet_to_dataframe(row, index, value, column, data):
    with io.BytesIO(value) as buffer:
        data[column.name] = (
            pyarrow.parquet.read_table(pyarrow.PythonFile(buffer, mode="r"))
            .to_pandas()
            .sort_values(by="epoch")
        )


column_converters[columns.get_index_of(run.run_history)] = parquet_to_dataframe
column_converters[columns.get_index_of(run.run_extended_history)] = parquet_to_dataframe


dfs = []

run vars {'_name': 'run', '_columns': (), '_index': None}
('experiment_id', 'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id', 'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory', 'host_name', 'batch', 'run_data', 'run_history', 'run_extended_history', 'queue', 'status', 'priority', 'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent', 'command')
{Column(_name='experiment_id', type_name='uuid'): 0, Column(_name='run_timestamp', type_name='timestamp'): 1, Column(_name='run_id', type_name='uuid'): 2, Column(_name='job_id', type_name='uuid'): 3, Column(_name='seed', type_name='bigint'): 4, Column(_name='slurm_job_id', type_name='bigint'): 5, Column(_name='task_version', type_name='smallint'): 6, Column(_name='num_nodes', type_name='smallint'): 7, Column(_name='num_cpus', type_name='smallint'): 8, Column(_name='num_gpus', type_name='smallint'): 9, Column(_name='gpu_memory', type_name='integer'): 10, Column(_name='host_name', type_name='text'): 11,

In [3]:
def get_ids():
    with ConnectionManager(credentials) as connection:
        """
        TODO: convert this into two steps:
        1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

            SELECT
                    {run}.{run_id}
            FROM
                    {run},
                    {job_status},
                    {job_data}
            WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                AND {job_status}.status = 2
            ORDER BY experiment_id, run_id;

        2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
        -> partition by the attributes we care about querying by (dataset, size, shape, depth)

                SELECT
                    {columns}
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {run}.{run_id} IN ({ids})

            pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
            results = pool.uimap(download_chunk, chunks)
            for num_rows, chunk in results:
                num_stored += 1
                print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

        extra credit) extract butter data matching this as well into a new dataset
        extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
        """
        query = psycopg.sql.SQL(
            """
                SELECT
                    {run}.run_id
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                ORDER BY experiment_id, run_id
                LIMIT 10;
            """ 
        ).format(
            run=run.identifier,
            job_status=job_status.identifier,
            job_data=job_data.identifier,
            pattern=sql.Literal("%energy%"),
        )

        with ClientCursor(connection) as c:
            print(c.mogrify(query))

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            ids = cursor.fetchall()
            rows = []
            for id in ids:
                rows.append(str(id[0]))
            return rows
        
rows = get_ids()



                SELECT
                    "run".run_id
                FROM
                    "run",
                    "job_status",
                    "job_data"
                WHERE TRUE
                    AND "run".batch like '%energy%'
                    AND "job_status".id = "run".run_id
                    AND "job_status".id = "job_data".id
                    AND "job_status".status = 2
                ORDER BY experiment_id, run_id
                LIMIT 10;
            


In [6]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
def get_data(run_id):
    print(run_id)
    with ConnectionManager(credentials) as connection:
        """
        TODO: convert this into two steps:
        1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

            SELECT
                    {run}.{run_id}
            FROM
                    {run},
                    {job_status},
                    {job_data}
            WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                AND {job_status}.status = 2
            ORDER BY experiment_id, run_id;

        2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
        -> partition by the attributes we care about querying by (dataset, size, shape, depth)

                SELECT
                    {columns}
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {run}.{run_id} IN ({ids})

            pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
            results = pool.uimap(download_chunk, chunks)
            for num_rows, chunk in results:
                num_stored += 1
                print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

        extra credit) extract butter data matching this as well into a new dataset
        extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
        """
        query = psycopg.sql.SQL(
            """
                SELECT
                    *
                FROM
                    run
                WHERE run_id = {run_id};
            """ 
        ).format(
            columns=columns.columns_sql,
            run_id=sql.Literal(run_id),
        )

        with ClientCursor(connection) as c:
            print(c.mogrify(query))

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            rows = cursor.fetchall()
            rows = [[str(item) for item in row] for row in rows]

            column_names = ["experiment_id", "run_timestamp", "run_id", "job_id", "seed", "slurm_job_id", "task_version", "num_nodes", "num_cpus", "num_gpus", "gpu_memory", "host_name", "batch", "run_data", "run_history", "run_extended_history", "task"]
            df = pd.DataFrame(rows, columns=column_names)

            # convert all columns to strings and drop "run_data", "run_history", "run_extended_history", "task"
            df = df.drop(columns=[ "run_data", "run_history", "run_extended_history", "task", "run_data", "run_history", "run_extended_history", "task"])
            print(df)

            table = pa.Table.from_pandas(df)
            pq.write_to_dataset(table, root_path='.', partition_cols=['batch', 'num_nodes', 'num_gpus', 'num_cpus'])


# run this in parallel
import multiprocessing.pool as pool
pool = pool.ThreadPool(10)
pool.map(get_data, rows)

34beb455-0bf6-44f7-99df-07b37589b9b50277ac37-8307-4d63-8e67-ecffcce04943

d3941040-8f04-4244-a9a2-9e706f244b93
81848207-6af1-4a51-a63a-27e05b9ab90e
dcc6f804-ee8f-4202-9e00-7c8494f63d3f
09de6a5a-48f0-4e63-a759-4c42e6c35d77
97320eaf-c8fb-49a9-a537-b6f3870808a2
249fb777-e48a-4656-b473-e6966c61d15f
943c5132-04c3-44b1-a048-aef110e31a4b
dc0e7b9e-c341-40e3-b889-d5fa9a86720f

                SELECT
                    *
                FROM
                    run
                WHERE run_id = '34beb455-0bf6-44f7-99df-07b37589b9b5';
            

                SELECT
                    *
                FROM
                    run
                WHERE run_id = '97320eaf-c8fb-49a9-a537-b6f3870808a2';
            

                SELECT
                    *
                FROM
                    run
                WHERE run_id = '81848207-6af1-4a51-a63a-27e05b9ab90e';
            

                SELECT
                    *
                FROM
                    run
              

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
data